In [ ]:
!pip install tensorflow==2.13 keras==2.13.1 onnx onnx-tf tf-keras

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = "./gpt2_tagalog"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
import torch
# Define input size (example: adjust to match your model)
dummy_input = torch.randint(0, tokenizer.vocab_size, (1, 64))  # Batch size 1, sequence length 64

# Export the model to ONNX
onnx_model_path = "tagalog_model.onnx"
torch.onnx.export(
    model, 
    dummy_input, 
    onnx_model_path,
    input_names=["input_ids"],
    output_names=["output"],
    dynamic_axes={"input_ids": {0: "batch_size", 1: "sequence_length"}},
    opset_version=14
)
print(f"Model successfully exported to ONNX at {onnx_model_path}")


In [ ]:
from onnx_tf.backend import prepare
import onnx

# Load ONNX model
onnx_model = onnx.load(onnx_model_path)

# Convert to TensorFlow SavedModel
saved_model_dir = "tagalog_model_tf"
tf_rep = prepare(onnx_model)
tf_rep.export_graph(saved_model_dir)
print(f"TensorFlow model saved to {saved_model_dir}")